In [1]:
%load_ext autoreload
%autoreload 2
import os
import random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
#import seaborn as sns
import sys
sys.path.append('../../')
from dataset import Dataset, SpikingDataset, RegSpikingDataset
from torch.utils.data.dataloader import DataLoader
from Loss import KDLoss
import torch
import torch.nn.functional as F
import torch.nn as nn
import torchvision
random.seed(1338)
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import copy
from pyESN import ESN
from scipy import interpolate
from gen_data import *
from tanh import tanh

from Loss import ber_loss


import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader

# import slayer from lava-dl
import lava.lib.dl.slayer as slayer

import IPython.display as display
from matplotlib import animation

generating_data = False

%load_ext autoreload

%autoreload 2

torch.__version__
# The coarse network structure is dicated by the Fashion MNIST dataset. 
dtype = torch.float

# Check whether a GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda:1')
    #device = torch.device("cuda")     
else:
    device = torch.device("cpu")
    
print(device)
tau_mem = 10e-3
tau_syn = 5e-3
time_step = 1e-3
alpha   = float(np.exp(-time_step/tau_syn))
beta    = float(np.exp(-time_step/tau_mem))

weight_scale = 7*(1.0-beta) # this should give us some spikes to begin with

print("init done")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cpu
init done


In [2]:
silent = True
method = 'RLS'  # RLS; INV; INV+RLS
# N_total_frame = 17
N_total_frame = 94
N_sync_frame = 4
# SNR_list = np.arange(1,20,2)
SNR_list = [55]

# Dataset selection
folder_name = 'data/S2/'  # LOS_Near:S2, LOS_Far:S3, NLOS:S1
output_folder = 'data_outputs/S1'

if folder_name == 'data/S1/':  # NLOS
    delay = 0
    packet_num = 21
elif folder_name == 'data/S2/':  # LOS_Near
    delay = 1
    packet_num = 27 # correct
elif folder_name == 'data/S3/':  # LOS_Far
    delay = 1
    packet_num = 22 # 23
else:
    print("Undefined Dataset")
    exit(1)
    
window_size = 2
N_reservoir = 16
debug = False

ber_record = []
dfe_ber_record = []
LS_ber_record = []
comb_ber_record = []
sta_ber_record = []
tanh_lut = tanh(
    input_bit=8,
    dx_bit=8,
    slope_fmt=(10, 10),
    intercept_fmt=(19, 19),
    max=8,
    better_lut=True,
    verbose=False,
    plot=False)

SNR = SNR_list[0]
i = 16
rc = RC(silent, method, N_total_frame, N_sync_frame, SNR, delay, window_size, i,
        N_reservoir=16,
        spectral_radius=0.2,
        sparsity=0.4,
        noise=1e-6,
        lut_activation=False,  # True,
        tanh_lut=tanh_lut,
        input_scale=25,  #40, #50, # 25,
        reservoir_input_scale = 8,  #4,  #5,
        show_wout=False,
        output_folder= output_folder,
        debug=debug,
        use_fpga= None,
        sock= None,  # usock
        addr = None) # addr

train_input, train_label, test_input, test_label = rc.run()
RC_test_input = np.load('gt_test_input_1.npy')
RC_train_input = np.load('gt_train_input_1.npy')
RC_train_label = np.load('gt_train_label_1.npy')

print(RC_test_input.shape, test_input.shape)
print("test_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_test_input), torch.tensor(test_input)))

print("train_input_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_input), torch.tensor(train_input)))

print("train_label_diff: ", torch.nn.MSELoss()(torch.tensor(RC_train_label), torch.tensor(train_label)))

train_mean = np.mean(train_input)
train_std = np.std(train_input)

train_input = (train_input - train_mean) / train_std
test_input = (test_input - train_mean) / train_std
train_label = 1.0 * train_label

FileNotFoundError: [Errno 2] No such file or directory: 'gt_test_input_1.npy'

In [ ]:
print(train_input.shape)
print(test_input.shape)
print(train_label.shape)
print(test_label.shape)

In [ ]:
import pandas as pd
import seaborn as sns


def pre_processing(train_input, train_label):
    idx_p = 10
    begin = 0 # N_total_frame * N_sync_frame
    
    # label index for data
    train_input_df = pd.DataFrame(train_input, columns = ['1','2', '3', '4'])
    #train_input_df['L1_idx'] = train_input_df.index % idx_p

    # label index for label
    train_label_df = pd.DataFrame(train_label, columns = ['L1','L2'])
    train_label_df['L1_idx'] = train_label_df.index % idx_p
    
    # split training and testing data
    test_input_df, test_label_df = train_input_df.iloc[75* 80 + 1:, :], train_label_df.iloc[75* 80 + 1:, :]
    train_input_df, train_label_df = train_input_df.iloc[:75* 80 + 1, :], train_label_df.iloc[:75* 80 + 1, :]

    # group by 
    #mapping = train_label_df.loc[begin:, :].groupby(by='L1_idx').mean().reset_index().loc[:, ['L1', 'L2', 'L1_idx']] 
    
    #train_input_df = pd.merge(train_input_df, mapping, how='left', on='L1_idx')

    #train_input_df = pd.get_dummies(train_input_df, prefix=['L'], columns=['L1_idx'])

    train_input_df = train_input_df.loc[begin:, :]


    print(train_input_df.head())
    
    # testing data
    # group by
    #test_input_df = test_input_df.merge(mapping, how = 'left', on='L1_idx')

    #test_input_df = pd.get_dummies(test_input_df, prefix=['L'], columns=['L1_idx'])

    print(test_input_df.head())

    train_input = train_input_df.to_numpy()
    test_input = test_input_df.to_numpy()
    
    train_label = train_label_df.drop(['L1_idx'], axis=1).to_numpy()
    test_label = test_label_df.drop(['L1_idx'], axis=1).to_numpy()
    
    print(train_input.shape)
    print(test_input.shape)
    
    
    return train_input, train_label, test_input, test_label

train_input, train_label, test_input, test_label = pre_processing(train_input, train_label)

In [3]:
nb_inputs  = 6
nb_steps  = 100

batch_size = 128

import scipy.io

from dataset import Dataset, RegTorchSeasonalitySpikingDataset, RegSpikingDataset, RegTorchSpikingDataset, RegTorchSeasonalityLinearSpikingDataset
train_data = RegTorchSeasonalityLinearSpikingDataset(train_input, train_label, nb_inputs, nb_steps)
test_data = RegTorchSeasonalityLinearSpikingDataset(test_input, test_label, nb_inputs, nb_steps)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
print(len(train_data))
print(train_data[0][0].shape)
print(train_data[0][1].shape)
print(train_data[0][2].shape)
print(train_data[0][3].shape)

7521
(24, 100)
(1,)
(24,)
(2,)


In [4]:
# generate fixed dataset
if generating_data:
    training_data = []
    testing_data = []
    for i in range(len(train_data)):
        X, X1, X_p, y = train_data[i]
        training_data.append([X, X1, X_p, y])
    
    for i in range(len(test_data)):
        X, X1, X_p, y = test_data[i]
        testing_data.append([X, X1, X_p, y])

    np.save('training_data.npy', training_data)
    np.save('testing_data.npy', testing_data)

In [20]:
"""
from dataset import AMySpikingDataset

training_data = np.load('training_data.npy', allow_pickle=True)
testing_data = np.load('testing_data.npy', allow_pickle=True)

train_data = AMySpikingDataset(training_data)
test_data = AMySpikingDataset(testing_data)
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
print(len(train_data))
print(train_data[0][0].shape)
print(train_data[0][1].shape)
print(train_data[0][2].shape)
print(train_data[0][3].shape)
"""

"\nfrom dataset import AMySpikingDataset\n\ntraining_data = np.load('training_data.npy', allow_pickle=True)\ntesting_data = np.load('testing_data.npy', allow_pickle=True)\n\ntrain_data = AMySpikingDataset(training_data)\ntest_data = AMySpikingDataset(testing_data)\ntrain_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)\ntest_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)\nprint(len(train_data))\nprint(train_data[0][0].shape)\nprint(train_data[0][1].shape)\nprint(train_data[0][2].shape)\nprint(train_data[0][3].shape)\n"

In [5]:
class Network(torch.nn.Module):
    def __init__(self, input_size, output_size):
        super(Network, self).__init__()

       # neuron_params = {
       #         'threshold'     : 0.1,
       #         'current_decay' : 1,
       #         'voltage_decay' : 0.1,
       #         'requires_grad' : True,     
       #     }
        #neuron_params_drop = {**neuron_params, 'dropout' : slayer.neuron.Dropout(p=0.05),}
        neuron_params = {
                'threshold'     : 2.0,
                'current_decay' : 0.25,
                'voltage_decay' : 0.03,
                'tau_grad'      : 0.03,
                'scale_grad'    : 3,
                'requires_grad' : True,   
            
            }
        
        
        self.blocks = torch.nn.ModuleList([
                #slayer.block.cuba.Input(neuron_params),
                #slayer.block.cuba.Recurrent(neuron_params, input_size, 8, weight_norm=True, delay=True),
                slayer.block.cuba.Dense(neuron_params, input_size, 8, weight_norm=True, delay=True, weight_scale=1.0),
                slayer.block.cuba.Dense(neuron_params, 8, 1, weight_norm=True, delay=True),
                #slayer.block.cuba.Dense(neuron_params, 8, 1, weight_norm=True, delay=True),
                #slayer.block.cuba.Dense(neuron_params, 128, output_size, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, input_size, 64, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 64, 128, weight_scale=2, weight_norm=True),
                #slayer.block.sigma_delta.Dense(sdnn_dense_params, 128, output_size, weight_scale=2, weight_norm=True)
                #slayer.block.cuba.Recurrent(cuba_params, 100, 50),
                #slayer.block.cuba.KWTA(cuba_params, 50, 50, num_winners=5)
            ])
    
    def forward(self, spike):
        for block in self.blocks:
            spike = block(spike)
        return spike

    def export_hdf5(self, filename):
        # network export to hdf5 format
        h = h5py.File(filename, 'w')
        layer = h.create_group('layer')
        for i, b in enumerate(self.blocks):
            b.export_hdf5(layer.create_group(f'{i}'))

class DNNNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 2)
        #self.fc2 = nn.Linear(16, 2)
        #self.act = nn.Tanh()

    def forward(self,x, x1, x2):
        x = x.flatten(start_dim=1)
        x = torch.cat((x, x2), axis=1)
        x = self.fc1(x)
        #x = self.act(x)
        #x = self.fc2(x)
        return x

In [6]:
from DFRSystem import *

net_snn = Network(nb_inputs * 4, 2).to(device)
net_dnn = DNNNetwork(1 * nb_steps + nb_inputs * 4 + 0 + 0 * nb_inputs, 2).to(device)
net_rc = TOriFloatDFRSystem(n_hidden=16, n_fc=64)

In [7]:
def test(test_loader, net_snn, net_dnn, rc, num_frame = 19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().detach().numpy())
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    #print("testing MSE: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(labels)))
    #print("testing MSE 0: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 0]), torch.tensor(labels[:, 0])))
    #print("testing MSE 1: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 1]), torch.tensor(labels[:, 1])))
    #print(all_output.shape, labels.shape)
    #print("percent loss: ", np.mean(np.abs(all_output - labels) / (np.abs(labels) + 1e-6)))
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    print("testing ber: ", rc.my_new_test(predict_time, target_time))


def test_teacher(test_loader, model, rc, nb_inputs, num_frame=19):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target.cpu().detach().numpy())
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
        
        input2 = input2.view(input2.shape[0], nb_inputs, 4)
        prev_out = 0.1 * torch.zeros(input2.size(0), model.n_hidden).to(device)
        output = model(input2, prev_out)[0].cpu().detach().numpy()
        all_output.append(output)
        
    
    all_output = np.concatenate(all_output, axis=0)
    labels = np.concatenate(labels, axis=0)
   
    #print("testing teacher MSE: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(labels)))
    #print("testing teacher MSE 0: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 0]), torch.tensor(labels[:, 0])))
    #print("testing teacher MSE 1: ", torch.nn.MSELoss()(torch.tensor(all_output[:, 1]), torch.tensor(labels[:, 1])))
    #print(all_output.shape, labels.shape)
    #print("percent teacher loss: ", np.mean(np.abs(all_output - labels) / (np.abs(labels) + 1e-6)))
    predict_time = rc.time_to_freq(all_output, num_frame, remove_delay=False)
    target_time = rc.time_to_freq(labels, num_frame, remove_delay=False)
    print("testing teacher ber: ", rc.my_new_test(predict_time, target_time))
    return rc.my_new_test(predict_time, target_time)

#test(test_loader, net_snn, net_dnn, rc, num_frame=19)
#test_teacher(test_loader, net_rc, rc, nb_inputs, num_frame=19)

In [8]:
def train_dnn(trainloader, testloader, net, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    optimizer = torch.optim.Adam(net.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    
    loss_fn = torch.nn.L1Loss()
    loss_hist = []
    best_ber = 10000000000.0
    
    for e in range(nb_epochs):
        print(e)
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer.zero_grad()
    
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net.n_hidden).to(device)
            output = net(x3_local, prev_out)[0]
            
            loss = loss_fn(output, y_local)
            
            loss.backward()
            optimizer.step()
            loss_hist.append(loss.item())
        print(np.mean(loss_hist))
        
        ber = test_teacher(test_loader, net, rc, nb_inputs, num_frame=19) 
        if ber < best_ber:
            best_ber = ber
            torch.save(net.state_dict(), './teacher.pt')

In [9]:
#train_dnn(train_loader, test_loader, net_rc, rc, nb_inputs, lr=1e-3, nb_epochs=100)

In [26]:
def mutual_loss(teacher_output, student_output, alpha, target, mode=True):
    # student loss 1
    loss_fn = torch.nn.L1Loss()
    
    #compare 
    loss_t = loss_fn(teacher_output, target)
    loss_s = loss_fn(student_output, target)
    
    loss1 = loss_fn(student_output, target)
    loss2 = loss_fn(student_output, teacher_output)
    if mode:
        if loss_t >= loss_s:
            return loss1
        else:
            return loss1 + alpha * loss2
    else:
        return (1 - alpha) * loss1 + alpha * loss2
    
def KD_train(trainloader, testloader, net_snn, net_dnn, net_teacher, rc, teacher_path, nb_inputs, lr=2e-3, nb_epochs=10):
    net_teacher.load_state_dict(torch.load(teacher_path))
    net_teacher.eval()
    test_teacher(testloader, net_teacher, rc, nb_inputs, num_frame=19)
    
    params = list(net_snn.parameters()) + list(net_dnn.parameters())
    optimizer_student = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    scheduler_student = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=200, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    loss_fn = torch.nn.L1Loss()
    loss_hist = []
    
    for e in range(nb_epochs):
        print(e)
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_student.zero_grad()
            
            # snn model 
            output = net_snn(x1_local)
            output = net_dnn(output, x2_local, x3_local)
            
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net_teacher.n_hidden).to(device)
           # teacher_output, _ = net_teacher(x3_local, prev_out)
            
            #student_loss = mutual_loss(teacher_output.detach(), output, alpha=0.0, target=y_local, mode=True)
            student_loss = loss_fn(output, y_local)
            student_loss.backward()

            optimizer_student.step()
        
        test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        #test_teacher(test_loader, net_teacher, rc, nb_inputs, num_frame=19)   

def mutual_train(trainloader, testloader, net_snn, net_dnn, net_teacher, rc, nb_inputs, lr=2e-3, nb_epochs=10):
    params = list(net_snn.parameters()) + list(net_dnn.parameters())
    optimizer_student = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    optimizer_teacher = torch.optim.Adam(net_teacher.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=1e-4)
    scheduler_student = torch.optim.lr_scheduler.StepLR(optimizer_student, step_size=100, gamma=0.1)
    scheduler_teacher = torch.optim.lr_scheduler.StepLR(optimizer_teacher, step_size=100, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    #loss_fn = torch.nn.SmoothL1Loss()
    loss_hist = []
    
    for e in range(nb_epochs):
        print(e)
        loss_hist = []
        for x1_local, x2_local, x3_local, y_local in trainloader:
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            x3_local = x3_local.float().to(device)
            y_local = y_local.float().to(device)
    
            optimizer_student.zero_grad()
            optimizer_teacher.zero_grad()
            
            # snn model 
            output = net_snn(x1_local)
            output = net_dnn(output, x2_local, x3_local)
            
            # lstm model
            x3_local = x3_local.view(x3_local.shape[0], nb_inputs, 4)
            prev_out = torch.zeros(x3_local.size(0), net_teacher.n_hidden).to(device)
            teacher_output, _ = net_teacher(x3_local, prev_out)
            
            student_loss = mutual_loss(teacher_output.detach(), output, alpha=0.05, target=y_local, mode=True)
            teacher_loss = mutual_loss(output.detach(), teacher_output, alpha=0.01, target=y_local, mode=False)
            student_loss.backward()
            teacher_loss.backward()
            optimizer_student.step()
            optimizer_teacher.step()
            loss_hist.append(student_loss.item() + teacher_loss.item())
        print(np.mean(loss_hist))
        
        test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        test_teacher(test_loader, net_teacher, rc, nb_inputs, num_frame=19)   

In [27]:
KD_train(train_loader, test_loader, net_snn, net_dnn, net_rc, rc, './teacher.pt', nb_inputs, lr=1e-3, nb_epochs=200)

/tmp/ipykernel_649/3724917593.py:41: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_649/3724917593.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_649/3724917593.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_649/3724917593.py:44: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_gra

testing teacher ber:  0.8103070175438597
0


/tmp/ipykernel_649/3724917593.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_649/3724917593.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_649/3724917593.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_649/3724917593.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad

testing ber:  0.4413377192982456
1
testing ber:  0.34868421052631576
2
testing ber:  0.2982456140350877
3
testing ber:  0.27028508771929827
4
testing ber:  0.22697368421052633
5
testing ber:  0.17269736842105263
6
testing ber:  0.1650219298245614
7
testing ber:  0.14692982456140352
8
testing ber:  0.13980263157894737
9
testing ber:  0.1299342105263158
10
testing ber:  0.11458333333333333
11
testing ber:  0.10526315789473684
12
testing ber:  0.09868421052631579
13
testing ber:  0.10197368421052631
14
testing ber:  0.09978070175438597
15
testing ber:  0.09923245614035088
16
testing ber:  0.09429824561403509
17
testing ber:  0.09703947368421052
18
testing ber:  0.0981359649122807
19
testing ber:  0.09594298245614036
20
testing ber:  0.10032894736842106
21
testing ber:  0.10471491228070176
22
testing ber:  0.10032894736842106
23


KeyboardInterrupt: 

In [8]:
net_snn = Network(nb_inputs * 4, 2).to(device)
net_dnn = DNNNetwork(1 * 100 + nb_inputs * 4 + 0 + 0 * nb_inputs, 2).to(device)
net_rc = TOriFloatDFRSystem(n_hidden=16, n_fc=64)

mutual_train(train_loader, test_loader, net_snn, net_dnn, net_rc, rc, nb_inputs, lr=1e-3, nb_epochs=100)

NameError: name 'mutual_train' is not defined

In [20]:
def new_loss(output, target):
    #loss = (output - target) ** 2 / (target ** 2 + 1e-6)
    return torch.mean(loss)

def train(trainloader, testloader, model, DNN_model, rc, lr=2e-3, nb_epochs=10):
    #params = [w1,w2]
    params = list(model.parameters()) + list(DNN_model.parameters())
    optimizer = torch.optim.Adam(params, lr=lr, betas=(0.9,0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.1)
    
    #loss_fn = slayer.loss.SpikeTime(time_constant=2, filter_order=2, reduction='mean').to(device)
    #loss_fn = torch.nn.MSELoss()
    loss_fn = torch.nn.L1Loss()
    #loss_fn = ber_loss
    loss_hist = []
    DNN_model.train()
    
    
    for e in range(nb_epochs):
        print(e)
        local_loss = []
        for x_local, x1_local, x2_local, y_local in trainloader:
            x_local = x_local.float().to(device)
            x1_local = x1_local.float().to(device)
            x2_local = x2_local.float().to(device)
            y_local = y_local.float().to(device)
    
            #output = model(x_local)
            #output = output.flatten(start_dim=1)
            optimizer.zero_grad()
            output = model(x_local)
            output = DNN_model(output, x1_local, x2_local)
            loss_val = loss_fn(output, y_local) 
            loss_val.backward()
            #print("AAAA: ", DNN_model.fc2.weight)
            #print("BBBB: ", DNN_model.fc1.weight.grad)
            optimizer.step()
            local_loss.append(loss_val.item())
            
        
        #if e % 1 == 30 and e != 0:
        #    print("Training accuracy: %.3f"%(compute_ber(trainloader, net, "train")))
        #    print("Test accuracy: %.3f"%(compute_ber(testloader, net, name)))
        scheduler.step()
        mean_loss = np.mean(local_loss)
        print(mean_loss)
        test(test_loader, net_snn, net_dnn, rc, num_frame=19)
        #print("Epoch %i: loss=%.5f"%(e+1,mean_loss))
        #test(testloader, model, DNN_model)
        #test_train(trainloader, model, DNN_model)

        

In [21]:
train(train_loader, test_loader, net_snn, net_dnn, rc, lr=1e-3, nb_epochs=100)

0
0.16545830064631523


/tmp/ipykernel_96/3983635306.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_96/3983635306.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_96/3983635306.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_96/3983635306.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(Tr

testing MSE:  tensor(0.0110, dtype=torch.float64)
testing MSE 0:  tensor(0.0098, dtype=torch.float64)
testing MSE 1:  tensor(0.0123, dtype=torch.float64)
percent loss:  2913.543207506088
testing ber:  0.29660087719298245
1
0.09254795027540084
testing MSE:  tensor(0.0037, dtype=torch.float64)
testing MSE 0:  tensor(0.0037, dtype=torch.float64)
testing MSE 1:  tensor(0.0036, dtype=torch.float64)
percent loss:  1424.6953153762524
testing ber:  0.21326754385964913
2
0.06021252561836166
testing MSE:  tensor(0.0014, dtype=torch.float64)
testing MSE 0:  tensor(0.0015, dtype=torch.float64)
testing MSE 1:  tensor(0.0013, dtype=torch.float64)
percent loss:  1141.5854933795567
testing ber:  0.09429824561403509
3
0.040663542562818275
testing MSE:  tensor(0.0014, dtype=torch.float64)
testing MSE 0:  tensor(0.0016, dtype=torch.float64)
testing MSE 1:  tensor(0.0012, dtype=torch.float64)
percent loss:  1167.347360962157
testing ber:  0.07182017543859649
4
0.03806374897744427
testing MSE:  tensor(0.00

KeyboardInterrupt: 

In [25]:
RC_train_time = np.load("gt_train_pred.npy")
print("train MSE: ", torch.nn.MSELoss()(torch.tensor(RC_train_time), torch.tensor(train_label)))
def test_train_both(train_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in train_loader:
        inputs.append(input)
        labels.append(target)
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
    
    all_output = np.concatenate(all_output, axis=0)
    print("train MSE RC: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(RC_train_time)))
    print("train MSE GT: ", torch.nn.MSELoss()(torch.tensor(all_output), torch.tensor(train_label)))
    print("train MSE GT: ", torch.nn.MSELoss()(torch.tensor(train_label), torch.tensor(RC_train_time)))

#train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
test_train_both(train_loader, net_snn, net_dnn)

train MSE:  tensor(0.0033, dtype=torch.float64)


/tmp/ipykernel_219/3403701246.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_219/3403701246.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_219/3403701246.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_219/3403701246.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_gra

train MSE RC:  tensor(0.0051, dtype=torch.float64)
train MSE GT:  tensor(0.0036, dtype=torch.float64)
train MSE GT:  tensor(0.0033, dtype=torch.float64)


In [10]:
net_snn.export_hdf5('./net_snn12.net')
torch.save(net_dnn.state_dict(), './net_dnn.pth')

TypeError: Object dtype dtype('O') has no native HDF5 equivalent

In [40]:
temp_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False, drop_last=False)
def test_MSE(test_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
    
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
        labels.append(target.cpu().detach().numpy())
    
    all_output = torch.tensor(np.concatenate(all_output, axis=0)).detach()
    labels = torch.tensor(np.concatenate(labels, axis=0)).detach()
    print(all_output.shape, labels.shape)
    loss_fn = torch.nn.MSELoss()
    
    print("all_loss: ", loss_fn(all_output, labels))
    print("no first 330 loss: ", loss_fn(all_output[330:, :], labels[330:, :]))

test_MSE(temp_loader, net_snn, net_dnn)

/tmp/ipykernel_98/229205384.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_98/229205384.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_98/229205384.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_98/229205384.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True)

torch.Size([7521, 2]) torch.Size([7521, 2])
all_loss:  tensor(0.0024)
no first 330 loss:  tensor(0.0014)


In [41]:
all_output = []
inputs = []
labels = []
test_loader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=False, drop_last=False)
def test(test_loader, net_snn, net_dnn):
    all_output = []
    inputs = []
    labels = []
    
    for input, input1, input2, target in test_loader:
        inputs.append(input)
        labels.append(target)
        target = torch.tensor(target).float()
        input = torch.tensor(input).float()
        input1 = torch.tensor(input1).float()
        input2 = torch.tensor(input2).float()
    
        output = net_snn(input)
        output = net_dnn(output, input1, input2).cpu().detach().numpy()
        all_output.append(output)
    
    all_output = np.concatenate(all_output, axis=0)
    ber = rc.my_test(all_output)
    print(ber)
    
test(test_loader, net_snn, net_dnn)

/tmp/ipykernel_98/3202473027.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).float()
/tmp/ipykernel_98/3202473027.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input = torch.tensor(input).float()
/tmp/ipykernel_98/3202473027.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input1 = torch.tensor(input1).float()
/tmp/ipykernel_98/3202473027.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(T

0.18902153558052434


In [11]:
A1 = 8 * torch.ones((128, 8))
A2 = torch.ones((128, 1))
print(A1 + A2)

tensor([[9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        ...,
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.],
        [9., 9., 9.,  ..., 9., 9., 9.]])
